<a href="https://colab.research.google.com/github/JamieBali/hopfieldSudokuSolver/blob/main/Hebbian_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hebbian Learning

Following "The Effects of Hebbian Learning on Optimisation in Hopfield Networks" (Watson, Buckley, and Mills), I intend to implement Hebbian Learning, and attempt to scale up the solution to solve Sudoku problems.

In [2]:
## imports
import random, math, pandas
import numpy as np
import matplotlib.pyplot as plt

## Models

The state of a Hopfield Network consisting of N discrete states $S_i$ where $i = 1,2,...,N$ can be written as $S = (s_i,...,s_N)$

$S_i(t+1) = Θ[∑_j ω_iⱼ S_i(t)]$
<br> Where ω are the elements of the connection matrix Ω, and Θ is the Heaviside threshold function.

The Hopfield network is run by repeatedly choosing a unit, $i$, uniformly at random and setting its state according to the above formula. 

Energy = -∑ᵢⱼωᵢⱼsᵢsⱼ


In [25]:
##
# The energy of a problem needs to be reduced. This function finds what the energy is at a given state.
##
def getEnergy(weights, neurones):
  #Energy = -sum(w_ij x s_i x s_j)
  sum = 0
  for i in range(0,len(neurones)):
    for j in range(0,len(neurones)):
      sum += (weights[i][j] * neurones[i] * neurones[j])
  energy = sum * -1
  return energy

##
# The heaviside function is an activation function that returns +1 for values >= 0, and -1 for values less than 0
##
def heaviside(value):
  if value < 0:
    return -1
  else:
    return 1

##
# The step function takes an index (i) for a given neurone to look at. This will be picked randomly.
##
def step(neurones, weights, i, size):
  # S_i (t+1) = Θ[sum(ω_ij * S_i (t)]

  sum = 0
  for j in range(0,len(neurones)):
    sum += neurones[j] * weights[i][j]

  neuroneCopy = np.copy(neurones)
  neuroneCopy[i] = heaviside(sum)
  #neuroneCopy[i] = neuroneCopy[i] * -1
  
  if getBoltzmanProbability(neurones, neuroneCopy, size) == 1:
    return neuroneCopy
  else:
    return neurones

## Boltzman Machine

The boltzman machine is a schocastic counterpart of the Hopfield network where a single state change is accepted probabilistically according to the change in energy it produces.

We can describe such a dynamical process more generally via a probability of accepting a stochatsic change to the system state:

$P[S(t+1) <- f(S(t))] = Θ'(ΔE)$ 

In [4]:
def getBoltzmanProbability(nt, ntplus, size = 9): # refering to weights at t+1, neurones at t+1, weights at t, and neurones at t respectively
  # t+1 - t = ΔE
  t1 = sudokuEnergy(ntplus, size)
  t = sudokuEnergy(nt, size)
  Delta = t1 - t  

  # below is a threshold function that takes values of 0 and 1 for negative and non-negative arguments respectively.
  return int(Delta <= 0)


# Random Restart

A random restart model is an option for avoiding the trap of a local optima, so the model can be programmed to periodically take a random state configuration, $R=\{-1|1\}ᴺ$, every τ steps.

After testing, the random restard causes too much fluctuation in this system and does not benefit the results.

In [5]:
def randomRestart(length):
  return random.choices([-1,1], k=length)

# Network Construction

In [16]:
# these three were taken randomly from sudoku.com
easy = [[0,0,0,0,7,9,0,3,0],[5,0,2,0,6,1,4,7,8],[3,7,6,0,8,5,9,0,2],[0,1,7,5,0,0,8,0,0],[2,0,9,8,3,0,0,0,0],[0,0,0,0,2,0,0,4,0],[0,0,0,0,5,0,2,0,1],[0,2,3,0,0,0,0,5,4],[1,0,0,7,0,0,0,0,0]]
medium = [[0,3,1,0,5,0,0,2,0],[0,0,0,0,0,2,9,0,5],[2,0,0,0,1,0,0,0,0],[3,5,0,0,9,0,0,7,0],[7,0,0,5,0,0,0,4,0],[0,1,0,7,0,3,2,0,0],[1,2,6,3,0,0,0,0,0],[0,9,0,8,0,5,0,0,0],[5,0,0,0,2,0,7,0,0]]
hard = [[0,4,0,0,0,5,0,6,0],[0,0,5,4,2,0,0,0,0],[0,0,1,6,0,3,5,0,4],[0,0,0,0,0,0,7,0,0],[0,3,7,0,0,0,0,1,0],[9,0,0,0,0,4,3,5,0],[0,0,4,2,5,0,0,0,0],[0,0,0,0,0,0,0,7,6],[6,0,9,0,7,0,0,0,5]]

# this is the solved version of the "easy" sudoku above.
solved = [[8,4,1,2,7,9,6,3,5],[5,9,2,3,6,1,4,7,8],[3,7,6,4,8,5,9,1,2],[4,1,7,5,9,6,8,2,3],[2,5,9,8,3,4,1,6,7],[6,3,8,1,2,7,5,4,9],[7,6,4,9,5,3,2,8,1],[9,2,3,6,1,8,7,5,4],[1,8,5,7,4,2,3,9,6]]

# this puzzle is taken from "expert star sudoku," a book of level 6 sudoku.
expert = [[0,0,0,0,1,0,0,0,0],[0,9,0,0,0,5,0,0,0],[5,0,0,0,7,6,0,0,0],[0,0,4,0,0,0,2,7,0],[0,2,0,0,0,0,5,0,9],[0,7,0,8,0,0,0,0,0],[0,0,1,0,0,3,0,0,0],[6,0,0,1,2,0,0,8,0],[7,8,0,0,0,0,9,0,0]]


In [158]:
###
# This converts the [0-9] construction of the puzzle into a [-1 | 1] construction which is more suitable for the network. 
###
def networkFormat(puzzle, size):
  neurones = []
  for row in range(0,size):
    for column in range(0, size):
      temp = []
      for x in range(0, size):
        temp.append(-1)
      num = puzzle[row][column]
      if num > 0:
        temp[num-1] = 1
      for x in temp:
        neurones.append(x)
  return neurones

###
# This converts the [-1 | 1] construction of the puzzle back into a [0-9] construction so it can be read. 
###
def readableFormat(neurones, size):
  puzzle = []
  temp = []
  for x in range(0,size**2):
    tile = neurones[x*size:((x+1)*size)]
    found = False
    for y in range(0,size):
      if tile[y] == 1:
        temp.append(y + 1)
        found = True
        break
    if found == False:
      temp.append(0)
    if (x + 1) % size == 0:
      puzzle.append(temp)
      temp = []
  return puzzle

In [19]:
def createWeights(neurones, size = 9):
  weights = []
  for x in range(0, len(neurones)):
    temp = []
    for y in range(0, len(neurones)):
      if y == x:
        temp.append(1)
      else:
        temp.append(-1)
    weights.append(temp)
  return weights

# Better Energy Functions

Using the hopfield energy will allow us to minimise the weight of a network, but in order to solve a puzzle we need to have our energy function represent how close the puzzle is to being solved. This *how-solved-ness* is an inverse score, such that 0 means a puzzle is perfectly solved.

This energy function, like the construction functions above, is written dynamically, so we can scale the size of the puzzles.

"In principle, the original behaviour of the system need not be governed by a weight matrix, but by some arbitrary energy function, $e(S)$, which is just some function of the state." (Watson, Buckley, and Mills)

In [101]:
def getLocation(row, column, number, size):
  return int((column * (size ** 2)) + (row * size) + number)
  

##
# This arbitrary energy function seeks to minimize the weight of a Sudoku problem. If the problem is perfectly solved, the energy should be 0.
# The energy will then be affected by the learned weights matrix.
##
def sudokuEnergy(neurones, size = 9):
  # For a completed 
  energy = 4*(size**2)

  # sum across i for all j,k (each number appears in each row once and only once)
  for column in range(0,size):
    for number in range(0,size):
      count = 0
      for row in range(0,size):
        if neurones[getLocation(row, column, number, size)] == 1:
          count += 1
      if count == 1:
        energy -= 1
      if count > 1:
        energy += 50 # a rule violation should lead to a massive increase in energy so it isn't picked

  # sum across j for all i,k (each number appears in each column once and only once)
  for row in range(0,size):
    for number in range(0,size):
      count = 0
      for column in range(0,size):
        if neurones[getLocation(row, column, number, size)] == 1:
          count += 1
      if count == 1:
        energy -= 1
      if count > 1:
        energy += 50

  # sum across k for all i,j (each tile contains only one number)
  for row in range(0,size):
    for cloumn in range(0,size):
      count = 0
      for number in range(0,size):
        if neurones[getLocation(row, column, number, size)] == 1:
          count += 1
      if count == 1:
        energy -= 1
      if count > 1:
        energy += 50

  # sum across i,j,k for each sub-grid on the board (each number appears in each sub-grid once and only once)
  temp = int(math.sqrt(size))
  for rowitt in range(0,temp):               # these row and cloumn itterators allows each individual sub-grid to be searched, and allows for easy grid size change
    for columnitt in range(0,temp):
      for number in range(0, size):
        sum = 0
        for row in range(0,temp):
          for column in range(0,temp):
            if neurones[getLocation((rowitt * temp) + row, (columnitt * temp) + column, number, size)] == 1:
              sum += 1
        if sum == 1:
          energy -= 1
        if sum > 1:
          energy += 50
  #if energy == 0:    # this is purely for debugging purposes.
  #  print("SOLVED!")
  
  return energy

# Theta Values

We can use theta as a value to represent given values at the start of the puzzle. We don't want these theta values to be changed, as they act as the constraints of the puzzle. We have 2 options for theta values - we can either update the neurones to factor in the theta values after any changes, or at the end of a relaxation period, or we can put in a prevention measure to stop the theta values from being changed in the first place. We will implement the latter solution.

In [65]:
def getThetaValues(neurones):
  theta = []
  for num, value in enumerate(neurones):
    if value == 1:
      theta.append(num)
  return theta

###
# We will construct a system to apply theta values, though we should not need to use it.
###
def applyTheta(neurones, theta, size):
  newNeurones = np.copy(neurones)
  for value in theta:
    flr = math.floor(value/size)      # by looking at the floor of val/size, we can make the 9 values to represent a tile equal -1
    for num in range(0, size):
      newNeurones[flr+num] = -1
    newNeurones[value] = 1            # we then set the correct tile to 1, establishing theta.
  return newNeurones

# Initial Testing

In order to test that the network is functioning, we have run the network on a 4x4 sudoku, in an attempt to find a solution to an easier puzzle. The network thus far is functionally a bit-flip algorithm with a simple activation function to only allow moves that would lower the overall energy.

In [154]:
four = [[0,3,4,0],[4,0,0,2],[1,0,0,3],[0,2,1,0]]
_pandasData = []

In [78]:
def runNetwork(puzzle, size, relaxation, epochs, verbose = False):
  neurones = networkFormat(puzzle, size)
  weights = createWeights(neurones, size)
  theta = getThetaValues(neurones)

  start = np.copy(neurones)
  bestEnergy = 10000
  bestNeurones = []

  for epoch in range(0, epochs):
    print(epoch)
    neurones = np.copy(start)
    for x in range(0,relaxation):
      i = random.randint(0,len(neurones)-1)
      if i in theta:          # this prevents any changes happening to the theta values in the network, ensuring the initial values are not modulated
        continue
      neurones = step(neurones, weights, i, size)
      if verbose and x % 10 == 0:
        print(readableFormat(neurones, size))
        print(sudokuEnergy(neurones, size))
      
    if sudokuEnergy(neurones, size) < bestEnergy:
      bestEnergy = sudokuEnergy(neurones,size)
      bestNeurones = neurones
    
    _pandasData.append(sudokuEnergy(neurones,size))
    weights = updateWeights(weights, neurones) # we only run this at the end of each relaxation period.
      

  return bestNeurones

# Hebbian Learning

By combining the functions written previously with a Hebbian Learning mechanism, we can update the weights of the network as we tend towards a solution. We want to update the weights with this system before we perform the random restart function, but only at the end of relaxation.

In [66]:
def updateWeights(weights, neurones, delta = 0.1): # where this delta value is a learning rate.
  for i in range(0, len(neurones)):
    for j in range(0, len(neurones)):
      if not(i == j) and neurones[i] == 1 and neurones[j] == 1:
        weights[i][j] += delta
  return weights

# Hebbian Testing

With the hebbian learning system implemented, we should see a general decrease in the energy between epochs, as the updated energies should allow the network to more quickly reach an optima.

In order to test this, we will run a 9x9 sudoku through the network, running for 250 steps, over 100 epochs.

In [ ]:
_pandasData = []
neurones = runNetwork(easy, 9, 1000, 300, False)
_pandasData